In [1]:
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.wcs
from IPython.display import clear_output

df1 = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e3.csv")
longitude_deg = 137.62535 # Kiso_observatory
latitude_deg = 35.7973
telescope_exposure_starting_time = np.arange(1).reshape(1,1)
ID = "d_fits/sub" + df1["ID"].values + ".fits" #fitヘッダーからexposure timeを引っ張る
for i in ID:
    hdul = astropy.io.fits.open(i)
    time = hdul[0].header["UTC"]
    telescope_exposure_starting_time = np.vstack((telescope_exposure_starting_time, time))
telescope_exposure_starting_time = np.delete(telescope_exposure_starting_time,0, 0)
df1["exposure_starting_time_tomoe"] = telescope_exposure_starting_time
df1["year_tomoe"] = df1["exposure_starting_time_tomoe"].str[:4]
df1["month_tomoe"] = df1["exposure_starting_time_tomoe"].str[5:7]
df1["day_tomoe"] = df1["exposure_starting_time_tomoe"].str[8:10]
df1["hh[UT]_tomoe"] = df1["exposure_starting_time_tomoe"].str[11:13]
df1["mm_tomoe"] = df1["exposure_starting_time_tomoe"].str[14:16]
df1["ss_tomoe"] = df1["exposure_starting_time_tomoe"].str[17:]
df1["year_tomoe"] = df1["year_tomoe"].astype(float)
df1["month_tomoe"] = df1["month_tomoe"].astype(float)
df1["day_tomoe"] = df1["day_tomoe"].astype(float)
df1["hh[UT]_tomoe"] = df1["hh[UT]_tomoe"].astype(float)
df1["mm_tomoe"] = df1["mm_tomoe"].astype(float)
df1["ss_tomoe"] = df1["ss_tomoe"].astype(float)
########################################################## ベクトルと radec
ra_dec1 = np.arange(2).reshape(1,2) #(ra,dec)
ra_dec2 = np.arange(2).reshape(1,2) #x-y平面でベクトル求めてradec,azelに変換
x_pixel = df1["X"].values
y_pixel = df1["Y"].values
x1_pixel = df1["x_1"].values #両端の点
for (i, j) in zip(x1_pixel, range(len(x1_pixel))):
    if i == 0.0:
        x1_pixel[j] = 1.0
x2_pixel = df1["x_2"].values
slope = np.tan(np.radians(90- df1["Angle"].values))
b = y_pixel + slope*x_pixel*(-1)
y1_pixel = slope*x1_pixel + b
y2_pixel = slope*x2_pixel + b

for (i, j, k, l, m) in zip(ID, x1_pixel, y1_pixel, x2_pixel, y2_pixel):
    fits = astropy.io.fits.open(i)
    wcs = astropy.wcs.WCS(fits[0].header, naxis = 2)
    ra_dec_1 = wcs.all_pix2world([[j, k]], 1)
    ra_dec_2 = wcs.all_pix2world([[l, m]], 1)
    ra_dec1 = np.vstack((ra_dec1, ra_dec_1))
    ra_dec2 = np.vstack((ra_dec2, ra_dec_2))
clear_output()
ra_dec1 = np.delete(ra_dec1,0, 0)
ra_dec2 = np.delete(ra_dec2,0, 0)
ra_dec = np.hstack((ra_dec1, ra_dec2))

columns = ["ra1[deg]", "dec1[deg]", "ra2[deg]", "dec2[deg]"]
radec = pd.DataFrame(data = ra_dec, columns = columns)
df1 = pd.concat([df1, radec], axis=1)
###########################################################################
##########################################################  ベクトルのradec to azel
floor_1 = 365.25*(df1["year_tomoe"] + 4716.0)
floor_2 = (30.6001*(df1["month_tomoe"] + 1.0) + 2.0)
floor_3 = df1["year_tomoe"]/100.0
floor_4 = (df1["year_tomoe"]/100.0)/4
floor_1 = floor_1.astype(int)
floor_2 = floor_2.astype(int)
floor_3 = floor_3.astype(int)
floor_4 = floor_4.astype(int)
JD = floor_1 + floor_2 - floor_3 + floor_4 +df1["day_tomoe"] - 1524.5 + \
            (df1["hh[UT]_tomoe"] + df1["mm_tomoe"]/60 + df1["ss_tomoe"]/3600)/24
D = JD - 2451543.5
w = 282.9404 + 4.70935e-5*D
M = (356.0470 + 0.9856002585*D)%360
L = w + M
GMST0 = ((L + 180)%360)/15
UT_hour = df1["hh[UT]_tomoe"] + df1["mm_tomoe"]/60 + df1["ss_tomoe"]/3600
SiderealTime = GMST0 + UT_hour + longitude_deg/15

HourAngle1 = (SiderealTime*15 - df1["ra1[deg]"])
HourAngle2 = (SiderealTime*15 - df1["ra2[deg]"])
A1 = np.cos(np.radians(HourAngle1))*np.cos(np.radians(df1["dec1[deg]"]))*np.cos(np.radians(90 - latitude_deg)) - \
    np.sin(np.radians(df1["dec1[deg]"]))*np.sin(np.radians(90 - latitude_deg))
A2 = np.cos(np.radians(HourAngle2))*np.cos(np.radians(df1["dec2[deg]"]))*np.cos(np.radians(90 - latitude_deg)) - \
    np.sin(np.radians(df1["dec2[deg]"]))*np.sin(np.radians(90 - latitude_deg))
B1 = np.sin(np.radians(HourAngle1))*np.cos(np.radians(df1["dec1[deg]"]))
B2 = np.sin(np.radians(HourAngle2))*np.cos(np.radians(df1["dec2[deg]"]))
C1 = np.cos(np.radians(HourAngle1))*np.cos(np.radians(df1["dec1[deg]"]))*np.sin(np.radians(90 - latitude_deg)) + \
    np.sin(np.radians(df1["dec1[deg]"]))*np.cos(np.radians(90 - latitude_deg))
C2 = np.cos(np.radians(HourAngle2))*np.cos(np.radians(df1["dec2[deg]"]))*np.sin(np.radians(90 - latitude_deg)) + \
    np.sin(np.radians(df1["dec2[deg]"]))*np.cos(np.radians(90 - latitude_deg))
az1 = (np.arctan(B1/A1)*180/np.pi) + 180
az2 = (np.arctan(B2/A2)*180/np.pi) + 180
el1 = np.arcsin(C1)*180/np.pi
el2 = np.arcsin(C2)*180/np.pi
azel = np.vstack((az1, el1, az2, el2))
azel = azel.transpose()
columns = ["az1[deg]", "el1[deg]", "az2[deg]", "el2[deg]"]
azel = pd.DataFrame(data = azel, columns = columns)
df1 = pd.concat([df1, azel], axis=1)
###########################################################################
#########################################################  modified mipointを求める
az1 = df1["az1[deg]"].values
el1 = df1["el1[deg]"].values
az2 = df1["az2[deg]"].values
el2 = df1["el2[deg]"].values
slope_tomoe = (el2 - el1)/(az2 - az1)
b_tomoe = el1 + slope_tomoe*az1*(-1)
az_middle = [0]
el_middle = [0]
dif_az = abs((az1 - az2)/2)
dif_el = abs((el1 - el2)/2)
for i in range(len(az1)):
    if az2[i] > az1[i]:
        az_mid =  az1[i] + dif_az[i]
        az_middle = np.vstack((az_middle, az_mid))
    else:
        az_mid = az2[i] + dif_az[i]
        az_middle = np.vstack((az_middle, az_mid))
    if el2[i] > el1[i]:
        el_mid =  el1[i] + dif_el[i]
        el_middle = np.vstack((el_middle, el_mid))
    else:
        el_mid = el2[i] + dif_el[i]
        el_middle = np.vstack((el_middle, el_mid))
az_middle = np.delete(az_middle, 0, 0)
el_middle = np.delete(el_middle, 0, 0)
azel_middle = np.hstack((az_middle, el_middle))
columns = ["az_middle[deg]", "el_middle[deg]"]
azel_middle = pd.DataFrame(data = azel_middle, columns = columns)
df1 = pd.concat([df1, azel_middle], axis=1)
#az_middle = np.reshape(az_middle_iccd, (len(az_middle_iccd),))
#el_middle = np.reshape(el_middle_iccd, (len(el_middle_iccd),))
###########################################################################
#########################################################  muのベクトル(az, el)を伸ばす
az1_mu = (df1["azimuth_start_at_kiso"]).values
el1_mu = (df1["elevation_start_at_kiso"]).values
az2_mu = (df1["azimuth_end_at_kiso"]).values
el2_mu = (df1["elevation_end_at_kiso"]).values
slope_mu = (el2_mu - el1_mu)/(az2_mu - az1_mu)
b_mu = el1_mu + slope_mu*az1_mu*(-1)
az3_mu = np.full(az1_mu.shape, 226.0)
az4_mu = np.full(az1_mu.shape, 242.0)
el3_mu = slope_mu*az3_mu + b_mu
el4_mu = slope_mu*az4_mu + b_mu

index = np.array(range(2000))
for (i, j) in zip(el3_mu, index):
    if i > 38.0:
        el3_mu[j] = 38.0
        az3_mu[j] = (38.0 - b_mu[j])/slope_mu[j]
    elif i < 23.0:
        el3_mu[j] = 23.0
        az3_mu[j] = (23.0 - b_mu[j])/slope_mu[j]

for (i, j) in zip(el4_mu, index):
    if i > 38.0:
        el4_mu[j] = 38.0
        az4_mu[j] = (38.0 - b_mu[j])/slope_mu[j]
    elif i < 23.0:
        el4_mu[j] = 23.0
        az4_mu[j] = (23.0 - b_mu[j])/slope_mu[j]
        
df1["az3[deg]_mu_at_kiso"] = pd.DataFrame(az3_mu)
df1["el3[deg]_mu_at_kiso"] = pd.DataFrame(el3_mu)
df1["az4[deg]_mu_at_kiso"] = pd.DataFrame(az4_mu)
df1["el4[deg]_mu_at_kiso"] = pd.DataFrame(el4_mu)
###########################################################################
#########################################################  modified midpointを通り
# muのベクトルへの最短距離を求める
az_midpoint_tomoe = (df1["az_middle[deg]"]).values
el_midpoint_tomoe = (df1["el_middle[deg]"]).values
slope_tomoe = (-1)*1/slope_mu
b_tomoe = el_midpoint_tomoe + slope_tomoe*az_midpoint_tomoe*(-1)
az_intersection = (b_mu - b_tomoe)/(slope_tomoe - slope_mu)
el_intersection = (slope_tomoe*b_mu - slope_mu*b_tomoe)/(slope_tomoe - slope_mu)
#modified midpointとintesection pointの距離を求める
a1_a2 = az_intersection - az_midpoint_tomoe
e1_e2 = el_intersection - el_midpoint_tomoe
a1 = np.power(a1_a2, 2)
e1 = np.power(e1_e2, 2)
d1 = a1 + e1
distance = np.sqrt(d1)
df1["az_intersection"] = pd.DataFrame(az_intersection)
df1["el_intersection"] = pd.DataFrame(el_intersection)
df1["distance"] = pd.DataFrame(distance)
df1["vector_az"] = df1["az_intersection"] - df1["az_middle[deg]"]
df1["vector_el"] = df1["el_intersection"] - df1["el_middle[deg]"]
###########################################################################
##################################################################  なす角求める
a = az1_mu - az2_mu #muのベクトル(a, b),長さ(l_mu)
b = el1_mu - el2_mu
a1 = np.power(a, 2)
b1 = np.power(b, 2)
l_mu = a1 + b1
l_mu = np.sqrt(l_mu)

c = az1 - az2 #tomo-eのベクトル(c, d),長さ(l_tomoe)
d = el1 - el2
c1 = np.power(c, 2)
d1 = np.power(d, 2)
l_tomoe = c1 + d1
l_tomoe = np.sqrt(l_tomoe)

#内積(inner_product),なす角(angle_between_tomoe_mu ; theta)
inner_product =  a*c + b*d
cos = inner_product/(l_mu*l_tomoe)
for (i, j) in zip(cos, range(len(cos))): #cosの値が負なら正になおす
    if i < 0.0:
        cos[j] = cos[j]*(-1)
theta = np.rad2deg(np.arccos(cos))
df1["theta"] = pd.DataFrame(theta)
df1.to_csv("simultaneous_event_candidates_table_MU+Tomo-e4.csv")
print "Complete!"

Complete!


In [2]:
#####################################################　　センサの視野を求める
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.wcs
from IPython.display import clear_output

longitude_deg = 137.62535 # Kiso_observatory
latitude_deg = 35.7973

df1 = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e4.csv")#センサの4角のra,decを求める
ra_dec1 = np.arange(2).reshape(1,2)
ra_dec2 = np.arange(2).reshape(1,2)
ra_dec3 = np.arange(2).reshape(1,2)
ra_dec4 = np.arange(2).reshape(1,2)
ID = "d_fits/sub" + df1["ID"].values + ".fits"
for i in ID: #  pixel to wcs
    fits = astropy.io.fits.open(i)
    wcs = astropy.wcs.WCS(fits[0].header, naxis = 2)
    ra_dec_1 = wcs.all_pix2world([[1.0, 1.0]], 1)
    ra_dec_2 = wcs.all_pix2world([[1.0, 282.0]], 1)
    ra_dec_3 = wcs.all_pix2world([[500.0, 282.0]], 1)
    ra_dec_4 = wcs.all_pix2world([[500.0, 1.0]], 1)
    ra_dec1 = np.vstack((ra_dec1, ra_dec_1))
    ra_dec2 = np.vstack((ra_dec2, ra_dec_2))
    ra_dec3 = np.vstack((ra_dec3, ra_dec_3))
    ra_dec4 = np.vstack((ra_dec4, ra_dec_4))
clear_output()
ra_dec1 = np.delete(ra_dec1,0, 0)
ra_dec2 = np.delete(ra_dec2,0, 0)
ra_dec3 = np.delete(ra_dec3,0, 0)
ra_dec4 = np.delete(ra_dec4,0, 0)
radec = np.hstack((ra_dec1, ra_dec2, ra_dec3, ra_dec4))
columns1 = ["ra1[deg]_sensor", "dec1[deg]_sensor", "ra2[deg]_sensor", "dec2[deg]_sensor",\
           "ra3[deg]_sensor", "dec3[deg]_sensor", "ra4[deg]_sensor", "dec4[deg]_sensor"]
df = pd.DataFrame(data = radec, columns = columns1)
df1 = pd.concat([df1, df], axis=1)
#######################################
#######################################################   ra,decをaz,elに変換
floor_1 = 365.25*(df1["year_tomoe"] + 4716.0)
floor_2 = (30.6001*(df1["month_tomoe"] + 1.0) + 2.0)
floor_3 = df1["year_tomoe"]/100.0
floor_4 = (df1["year_tomoe"]/100.0)/4
floor_1 = floor_1.astype(int)
floor_2 = floor_2.astype(int)
floor_3 = floor_3.astype(int)
floor_4 = floor_4.astype(int)
JD = floor_1 + floor_2 - floor_3 + floor_4 +df1["day_tomoe"] - 1524.5 + \
            (df1["hh[UT]_tomoe"] + df1["mm_tomoe"]/60 + df1["ss_tomoe"]/3600)/24
D = JD - 2451543.5
w = 282.9404 + 4.70935e-5*D
M = (356.0470 + 0.9856002585*D)%360
L = w + M
GMST0 = ((L + 180)%360)/15
UT_hour = df1["hh[UT]_tomoe"] + df1["mm_tomoe"]/60 + df1["ss_tomoe"]/3600
SiderealTime = GMST0 + UT_hour + longitude_deg/15

HourAngle_1 = (SiderealTime*15 - df1["ra1[deg]_sensor"])
A_1 = np.cos(np.radians(HourAngle_1))*np.cos(np.radians(df1["dec1[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg)) - \
    np.sin(np.radians(df1["dec1[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg))
B_1 = np.sin(np.radians(HourAngle_1))*np.cos(np.radians(df1["dec1[deg]_sensor"]))
C_1 = np.cos(np.radians(HourAngle_1))*np.cos(np.radians(df1["dec1[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg)) + \
    np.sin(np.radians(df1["dec1[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg))

HourAngle_2 = (SiderealTime*15 - df1["ra2[deg]_sensor"])
A_2 = np.cos(np.radians(HourAngle_2))*np.cos(np.radians(df1["dec2[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg)) - \
    np.sin(np.radians(df1["dec2[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg))
B_2 = np.sin(np.radians(HourAngle_2))*np.cos(np.radians(df1["dec2[deg]_sensor"]))
C_2 = np.cos(np.radians(HourAngle_2))*np.cos(np.radians(df1["dec2[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg)) + \
    np.sin(np.radians(df1["dec2[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg))

HourAngle_3 = (SiderealTime*15 - df1["ra3[deg]_sensor"])
A_3 = np.cos(np.radians(HourAngle_3))*np.cos(np.radians(df1["dec3[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg)) - \
    np.sin(np.radians(df1["dec3[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg))
B_3 = np.sin(np.radians(HourAngle_3))*np.cos(np.radians(df1["dec3[deg]_sensor"]))
C_3 = np.cos(np.radians(HourAngle_3))*np.cos(np.radians(df1["dec3[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg)) + \
    np.sin(np.radians(df1["dec3[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg))

HourAngle_4 = (SiderealTime*15 - df1["ra4[deg]_sensor"])
A_4 = np.cos(np.radians(HourAngle_4))*np.cos(np.radians(df1["dec4[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg)) - \
    np.sin(np.radians(df1["dec4[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg))
B_4 = np.sin(np.radians(HourAngle_4))*np.cos(np.radians(df1["dec4[deg]_sensor"]))
C_4 = np.cos(np.radians(HourAngle_4))*np.cos(np.radians(df1["dec4[deg]_sensor"]))*np.sin(np.radians(90 - latitude_deg)) + \
    np.sin(np.radians(df1["dec4[deg]_sensor"]))*np.cos(np.radians(90 - latitude_deg))
az_1 = (np.arctan(B_1/A_1)*180/np.pi) + 180
el_1 = np.arcsin(C_1)*180/np.pi
az_2 = (np.arctan(B_2/A_2)*180/np.pi) + 180
el_2 = np.arcsin(C_2)*180/np.pi
az_3 = (np.arctan(B_3/A_3)*180/np.pi) + 180
el_3 = np.arcsin(C_3)*180/np.pi
az_4 = (np.arctan(B_4/A_4)*180/np.pi) + 180
el_4 = np.arcsin(C_4)*180/np.pi

azel = np.vstack((az_1, el_1, az_2, el_2, az_3, el_3, az_4, el_4))
azel = azel.transpose()
columns = ["az1[deg]_sensor", "el1[deg]_sensor", "az2[deg]_sensor", "el2[deg]_sensor", "az3[deg]_sensor",\
             "el3[deg]_sensor", "az4[deg]_sensor", "el4[deg]_sensor"]
azel = pd.DataFrame(data=azel, columns=columns)
df2 = df1.loc[:,["ID","theta","distance"]]
df3 = pd.concat([df2, azel], axis=1)
df3.to_csv("sensor.csv")
print "Complete!"

Complete!
